In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)


In [1]:
import minepy

ImportError: numpy.core.multiarray failed to import

In [33]:
dfAll = pd.read_csv('../../Data/All_Data_dedup_zero.csv')

In [34]:
for var in dfAll.columns:
    if dfAll[var].dtypes!='object':
        if dfAll[var].nunique()==1:
            print(var)
            del dfAll[var]

210X21
210X59
210X77
210X199
210X218
210X222
210X225
220X485
220X551
220X557


In [4]:
feat_list = dfAll.columns.tolist()[14:]
dfAll_tmp = dfAll.loc[dfAll['ID']!='ID454']
dfTmp = dfAll_tmp[feat_list]
dfTmp.shape

(720, 3207)

In [32]:
zero_cnt = (dfTmp==0).sum().to_dict()
zero_cnt2 = {}
for key, value in zero_cnt.items():
    if not zero_cnt2.__contains__(value) and value!=0:
        zero_cnt2[value] =[key]
    elif value!=0:
        zero_cnt2[value].append(key)

In [35]:
dfTmp2 = dfTmp.copy(deep=True)
for key, value in zero_cnt2.items():
    if len(value)<2:
        continue
    check = 1
    ind = dfTmp2.loc[dfTmp2[value[0]]==0].index.tolist()
    for var in value[1:]:
        if var=='400X141':
            continue
        if ind!=dfTmp2.loc[dfTmp2[var]==0].index.tolist():
            check=0
            break
    if check==1:
        print(key,value)
        dfTmp.loc[dfTmp[value[0]]==0,value]=np.nan
        

40 ['210X1', '210X2', '210X3', '210X4', '210X5', '210X6', '210X11', '210X12', '210X39', '210X52', '210X209', '210X210']


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


11 ['210X13', '210X14', '210X15', '210X16', '210X25', '210X26', '210X27', '210X28', '210X32', '210X33', '210X34', '210X35', '210X40', '210X41', '210X42', '210X43', '210X44', '210X45', '210X46', '210X47', '210X48', '210X49', '210X50', '210X51', '210X53', '210X54', '210X55', '210X56', '210X57', '210X58', '210X59', '210X60', '210X61', '210X62', '210X63', '210X64', '210X65', '210X66', '210X67', '210X68', '210X69', '210X70', '210X71', '210X72', '210X73', '210X74', '210X75', '210X76', '210X81', '210X82', '210X83', '210X84', '210X85', '210X86', '210X87', '210X88', '210X112', '210X113', '210X114', '210X115', '210X116', '210X117', '210X118', '210X119', '210X120', '210X121', '210X122', '210X123', '210X124', '210X125', '210X126', '210X127', '210X128', '210X129', '210X130', '210X131', '210X132', '210X133', '210X134', '210X135', '210X136', '210X137', '210X138', '210X139', '210X140', '210X141', '210X142', '210X143', '210X144', '210X145', '210X146', '210X147', '210X148', '210X149', '210X150', '210X15

In [45]:
dfAll.loc[dfAll['ID']!='ID454',feat_list] = dfTmp[feat_list]

In [51]:
dfAll.loc[dfAll['ID']=='ID454',var]==0

303    True
Name: 210X1, dtype: bool

In [57]:
for var in feat_list:
    if ((dfAll[var]==0).sum())==1 and dfAll.loc[dfAll['ID']=='ID454',var].values[0]==0:
        print(var)
        dfAll.loc[dfAll['ID']=='ID454',var]=np.nan

210X1
210X2
210X3
210X4
210X5
210X6
210X10
210X11
210X12
210X13
210X14
210X15
210X16
210X19
210X20
210X21
210X23
210X25
210X26
210X27
210X28
210X32
210X33
210X34
210X35
210X36
210X39
210X40
210X41
210X42
210X43
210X44
210X45
210X46
210X47
210X48
210X49
210X50
210X51
210X52
210X53
210X54
210X55
210X56
210X57
210X58
210X59
210X60
210X61
210X62
210X63
210X64
210X65
210X66
210X67
210X68
210X69
210X70
210X71
210X72
210X73
210X74
210X75
210X76
210X77
210X81
210X82
210X83
210X84
210X85
210X86
210X87
210X88
210X89
210X90
210X91
210X92
210X93
210X94
210X95
210X96
210X100
210X101
210X102
210X104
210X105
210X106
210X107
210X108
210X110
210X112
210X113
210X114
210X115
210X116
210X117
210X118
210X119
210X120
210X121
210X122
210X123
210X124
210X125
210X126
210X127
210X128
210X129
210X130
210X131
210X132
210X133
210X134
210X135
210X136
210X137
210X138
210X139
210X140
210X141
210X142
210X143
210X144
210X145
210X146
210X147
210X148
210X149
210X150
210X151
210X153
210X154
210X155
210X156
210X157
210X158

In [58]:
dfAll.to_csv('../../Data/All_Data_dedup_zero.csv',index=False)

In [24]:
dfTmp2 = dfTmp.copy(deep=True)
for key, value in zero_cnt2.items():
    if len(value)<2:
        continue
    check = 1
    ind = dfTmp2.loc[dfTmp2[value[0]]==0].index.tolist()
    for var in value[1:]:
        if ind!=dfTmp2.loc[dfTmp2[var]==0].index.tolist():
            check=0
            break
    if check==1:
        print(key,value)
        dfTmp.loc[dfTmp[value[0]]==0,value]=np.nan
        

73 ['210X228', '210X229', '210X230', '210X231']


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


371 ['220X289', '220X485']
208 ['220X300', '220X348', '220X496']
509 ['220X526', '220X554']
712 ['220X551', '220X557']
424 ['310X19', '310X20', '310X21', '310X22', '310X23', '310X24']
257 ['311X66', '311X154', '261X340', '261X526', '261X712']
162 ['311X69', '311X157', '261X343', '261X529', '261X715']
77 ['330X35', '330X38', '330X101', '330X103', '330X627', '330X629', '330X631', '330X634', '330X749']
6 ['344X237', '344X285']
715 ['344X270', '344X306']


In [37]:
dfAll2 = pd.read_csv('../../Data/All_Data_dedup.csv')

In [41]:
'''for var in dfAll.columns:
    if 'TOOL' in var:
        del dfAll[var]'''
dfAll3 = dfAll.merge(dfAll2.iloc[:,0:14],'inner','ID')
dfAll3.shape

(721, 3231)

In [35]:
for var in dfAll.columns:
    if 'TOOL' in var:
        tmpTrain = pd.get_dummies(dfAll[var],prefix=var,dummy_na=True)
        dfAll = pd.concat([dfAll,tmpTrain],axis=1)
        del dfAll[var]

In [36]:
tmp = dfAll.isnull().sum().to_dict()
null_dict ={}
for key,value in tmp.items():
    if key=='ID':
        continue
    if not null_dict.__contains__(value):
        null_dict[value] = [key]
    else:
        null_dict[value].append(key)

In [37]:
keys = list(null_dict.keys())
keys.sort()
keys

[0,
 1,
 3,
 4,
 5,
 6,
 7,
 8,
 12,
 19,
 21,
 29,
 30,
 35,
 36,
 41,
 71,
 73,
 77,
 162,
 257,
 348,
 366,
 375,
 513,
 715]

In [38]:
for key,value in null_dict.items():
    print(key,len(value))

41 13
0 2137
1 56
12 129
73 4
4 245
375 1
513 2
30 2
35 1
36 11
21 1
71 8
348 1
257 5
162 5
366 72
77 9
6 3
715 2
5 117
3 115
19 82
8 120
7 1
29 140


In [39]:
used_list = null_dict[0]
for key in keys:
    if key==0:
        continue
    for var in null_dict[key]:
        dfAll_input = dfAll[used_list]
        train = dfAll_input.loc[dfAll[var].notnull()]
        target = dfAll.loc[dfAll[var].notnull(),var]
        test = dfAll_input.loc[dfAll[var].isnull()]
        model = RandomForestRegressor(n_estimators=2000, max_features='sqrt',  max_depth=4, random_state=19900615)  
        model.fit(train,target)
        predicted = model.predict(test)

        if dfAll[var].nunique()<=10:
            print('Category variable')
            tmp_unique = dfAll.loc[dfAll[var].notnull(),var].unique()
            for i in range(len(predicted)):
                predicted[i] = tmp_unique[np.abs(predicted[i]-tmp_unique)==np.abs(predicted[i]-tmp_unique).min()][0]
        dfAll.loc[dfAll[var].isnull(),var] = predicted
        print(var)
        used_list.append(var)

210X10
Category variable
210X19
210X20
210X23
Category variable
210X36
210X89
210X90
210X91
Category variable
210X92
Category variable
210X93
210X94
210X95
210X96
210X100
210X101
Category variable
210X102
210X104
210X105
210X106
210X107
Category variable
210X108
Category variable
210X110
210X184
210X185
210X186
210X187
210X188
210X189
210X190
210X192
210X194
210X195
210X206
210X207
Category variable
210X214
210X216
210X217
210X219
Category variable
210X220
Category variable
210X226
220X26
220X27
220X28
220X29
220X30
220X31
220X32
220X33
Category variable
220X53
220X54
220X55
Category variable
220X67
220X107
Category variable
220X108
Category variable
220X223
220X224
Category variable
420X1
420X4
420X12
420X13
420X14
420X15
420X16
Category variable
420X19
Category variable
420X20
Category variable
420X21
Category variable
420X22
Category variable
420X23
420X30
420X31
420X32
420X33
Category variable
420X34
Category variable
420X35
Category variable
420X36
Category variable
420X37
Categor

Category variable
400X187
Category variable
400X188
Category variable
400X189
400X192
400X193
400X194
400X195
400X196
Category variable
400X197
Category variable
400X198
400X199
400X200
Category variable
400X201
Category variable
400X202
Category variable
400X204
400X205
400X208
400X209
400X210
400X211
400X212
400X213
400X214
400X215
400X216
400X217
Category variable
400X228
344X237
344X285
Category variable
520X174
520X345
Category variable
520X140
520X143
520X158
520X159
520X160
520X161
520X162
Category variable
520X165
Category variable
520X166
Category variable
520X167
Category variable
520X168
Category variable
520X169
520X176
520X177
520X178
520X179
Category variable
520X180
Category variable
520X181
Category variable
520X182
Category variable
520X183
Category variable
520X184
Category variable
520X186
520X230
520X231
520X232
520X233
520X234
Category variable
520X235
520X236
520X237
520X238
520X239
520X240
520X241
520X242
520X243
520X244
520X245
520X252
520X253
520X254
520X255
52

312X115
312X121
312X127
312X133
312X139
312X145
Category variable
312X163
Category variable
312X169
Category variable
312X175
Category variable
312X187
Category variable
312X193
Category variable
312X199
Category variable
312X211
Category variable
312X217
Category variable
312X223
Category variable
312X235
Category variable
312X240
312X246
Category variable
312X257
Category variable
312X263
Category variable
312X269
Category variable
312X281
Category variable
312X287
Category variable
312X293
Category variable
312X311
Category variable
312X335
Category variable
312X341
Category variable
312X353
Category variable
312X359
Category variable
312X365
Category variable
312X377
Category variable
312X383
Category variable
312X389
Category variable
312X430
Category variable
312X436
Category variable
312X442
Category variable
312X448
Category variable
312X478
Category variable
312X520
Category variable
312X574
Category variable
312X580
Category variable
312X586
Category variable
312X592
Category

In [40]:
dfAll.to_csv('../../Data/All_non_missing_v2.csv',index=False)

In [10]:
dfAll.shape

(721, 3293)

In [2]:
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

Load data...
Done.


In [3]:
#load features
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')
   
        


In [ ]:
##define the variables
'''var_change = {'raw':raw}
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv',nrows=10)
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    var_change[filename] = {'constant':[],'category':[]}
    for var in tmp_feat_list:
        if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
            var_change[filename]['constant'].append(var)
        elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
            var_change[filename]['category'].append(var)
'''



In [4]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
feat_cnt = 0
for key,value in var_change.items():
    feat_cnt+=1
    if feat_cnt>1:
        continue
    for var in var_change[key]['constant']:
        predictors.remove(var)
    for var in var_change[key]['category']:
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

In [26]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5, params = {'max_depth':4, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':2,'subsample':0.8,
                         'colsample_bytree':0.8}):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = params 
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =1000)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp,metrics.mean_squared_error(test_result['target'], test_result['score'])

In [ ]:
record = []
for md in [3,4,5]:
    for la in [0.8,1,1.3,1.5,2]:
        for sp in [0.6,0.8,1]:
            for cb in [0.2,0.4,0.6,0.8,1]:
                tmp_record = [md,la,sp,cb]
                print(tmp_record)
                params = {'max_depth':md, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':la,'subsample':sp,'colsample_bytree':cb}
                tmp_record.append(xgb_kfold(dfTrain,dfPred,predictors,5,params=params)[3])
                record.append(tmp_record)
                GS_df = pd.DataFrame(record,columns=['max_depth','lambda','subsample','colsample_bytree','test_MSE'])
                GS_df.to_csv('../../Profiling/GS_result.csv',index=False)

[3, 0.8, 0.6, 0.2]
[0]	train-rmse:2.33435	eval-rmse:2.31993
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[508]	train-rmse:0.080587	eval-rmse:0.17733

[0]	train-rmse:2.32603	eval-rmse:2.35314
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[1000]	train-rmse:0.036267	eval-rmse:0.190875
Stopping. Best iteration:
[964]	train-rmse:0.038393	eval-rmse:0.19084

[0]	train-rmse:2.3309	eval-rmse:2.33339
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
Stopping. Best iteration:
[626]	train-rmse:0.067672	eval-rmse:0.14085

[0]	train-rmse:2.32928	eval-rmse:2.34094
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved i

In [12]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33325	eval-rmse:2.32446
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42504	eval-rmse:1.41672
[100]	train-rmse:0.879138	eval-rmse:0.872982
[150]	train-rmse:0.55208	eval-rmse:0.555279
[200]	train-rmse:0.355857	eval-rmse:0.372674
[250]	train-rmse:0.236805	eval-rmse:0.272561
[300]	train-rmse:0.163235	eval-rmse:0.220306
[350]	train-rmse:0.117279	eval-rmse:0.19572
[400]	train-rmse:0.088505	eval-rmse:0.184687
[450]	train-rmse:0.069554	eval-rmse:0.180748
[500]	train-rmse:0.056817	eval-rmse:0.178809
[550]	train-rmse:0.047473	eval-rmse:0.17801
[600]	train-rmse:0.04041	eval-rmse:0.177439
[650]	train-rmse:0.034622	eval-rmse:0.17706
[700]	train-rmse:0.029927	eval-rmse:0.177028
[750]	train-rmse:0.025907	eval-rmse:0.176934
[800]	train-rmse:0.022595	eval-rmse:0.176859
[850]	train-rmse:0.019805	eval-rmse:0.176851
Stopping. Best iteration:
[819]	train-rmse:0.021476	eval-rm

In [13]:
imp.to_csv('../../Profiling/imp_test_v3.csv',index=False)

In [ ]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)

In [ ]:
#貌似RF根本就不需要CV啊，其实也行吧
def rf_kfold(dfTrain,dfPred,predictors,n_splits=5):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors].values
        test_X = dfTrain.loc[test_index,predictors].values
        train_Y = dfTrain.loc[train_index,'Y'].values
        test_Y = dfTrain.loc[test_index,'Y'].values
 
        model = RandomForestRegressor(n_estimators=1999, max_features='sqrt', max_depth=None, max_depth=4, compute_importances=True, random_state=615)  
        model.fit(train_X,train_Y)
        pred_test = model.predict(test_X）
        pred_score = model.predict(dfPred[predictors].values)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp,metrics.mean_squared_error(test_result['target'], test_result['score'])

In [ ]:
#RF填缺失值